# Indmat prosjekt
###### Liva Berge Flo, André Pettersen-Dahl, Herman Neple

#### Oppgave 1

#### Oppgave 3

Vi begynner med å importere alle bibliotek vi trenger

In [1]:
from layers import *
from neural_network import NeuralNetwork
from utils import onehot
import numpy as np
from data_generators import get_train_test_sorting

In [13]:
#definerer variabler
r = 5
m = 2

d = 10
k = 5
p = 15
L = 2

embed = EmbedPosition(9,m,d)
att1 = Attention(d,k)
ff1 = FeedForward(d,p)
un_embed = LinearLayer(d,m)
softmax = Softmax()
loss = CrossEntropy()

nn = NeuralNetwork([embed, att1, ff1, un_embed, softmax])

data = get_train_test_sorting(r, m, samples_per_batch=250,n_batches_train=10, n_batches_test=4)

x_train = data['x_train']
y_train = data['y_train']

for j in range(50):
    losses = []
    for i in range(10):
        X = onehot(x_train[i], m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y_train[i]))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_Adam()
    mean_loss = np.mean(losses)
    print(f'Iterasjon {j} L = {mean_loss}')

print()

Iterasjon 0 L = 0.6787757717797118
Iterasjon 1 L = 0.5932622062438606
Iterasjon 2 L = 0.49316525545713874
Iterasjon 3 L = 0.4296231648022152
Iterasjon 4 L = 0.41298370917212396
Iterasjon 5 L = 0.42361285520350345
Iterasjon 6 L = 0.4273918446238524
Iterasjon 7 L = 0.4030138198422872
Iterasjon 8 L = 0.39548984166230233
Iterasjon 9 L = 0.38907205200523454
Iterasjon 10 L = 0.35513256103956914
Iterasjon 11 L = 0.33770758503073595
Iterasjon 12 L = 0.3292865298470031
Iterasjon 13 L = 0.3036124651486691
Iterasjon 14 L = 0.31247154065890803
Iterasjon 15 L = 0.34544068206104966
Iterasjon 16 L = 0.3721613906255545
Iterasjon 17 L = 0.4375538813024608
Iterasjon 18 L = 0.40552830832555936
Iterasjon 19 L = 0.37050402832921375
Iterasjon 20 L = 0.3169985159340846
Iterasjon 21 L = 0.30216575641827303
Iterasjon 22 L = 0.2903403607675081
Iterasjon 23 L = 0.2858681670007307
Iterasjon 24 L = 0.28316623349054326
Iterasjon 25 L = 0.2827593831271095
Iterasjon 26 L = 0.28377031877931713
Iterasjon 27 L = 0.28210

In [14]:
for j in range(50):
    losses = []
    for i in range(10):
        X = onehot(x_train[i], m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y_train[i]))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_Adam()
    mean_loss = np.mean(losses)
    print(f'Iterasjon {j} L = {mean_loss}')

Iterasjon 0 L = 0.27845014337759755
Iterasjon 1 L = 0.27839109602308104
Iterasjon 2 L = 0.27881457669775483
Iterasjon 3 L = 0.2805350875476408
Iterasjon 4 L = 0.28107260748446883
Iterasjon 5 L = 0.28271137300064086
Iterasjon 6 L = 0.280553740420599
Iterasjon 7 L = 0.2805931628770747
Iterasjon 8 L = 0.278835438816442
Iterasjon 9 L = 0.2781063545256889
Iterasjon 10 L = 0.27761818813151623
Iterasjon 11 L = 0.27782997176638063
Iterasjon 12 L = 0.27698394582095787
Iterasjon 13 L = 0.2775253162759326
Iterasjon 14 L = 0.276451015435799
Iterasjon 15 L = 0.27673866365765654
Iterasjon 16 L = 0.2759591250940643
Iterasjon 17 L = 0.2762780235337985
Iterasjon 18 L = 0.2777416392714727
Iterasjon 19 L = 0.276466353726886
Iterasjon 20 L = 0.2774564240898313
Iterasjon 21 L = 0.27698769697867254
Iterasjon 22 L = 0.2773126565135799
Iterasjon 23 L = 0.2782033686471612
Iterasjon 24 L = 0.27732078841364277
Iterasjon 25 L = 0.278501282099973
Iterasjon 26 L = 0.2774253513802297
Iterasjon 27 L = 0.2789022140596

In [26]:
print(np.argmax(nn.forward(onehot(np.array([[1, 0 ,0, 0, 1]]), m)),axis=1))

[[1 1 0 0 0]]
